## Import orchestrator and initialize it

In [1]:
from data_gatherer.orchestrator import Orchestrator
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
data_gatherer = Orchestrator()

## Feed some input data to the orchestrator

In [4]:
input_data = [
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6141466",
    "https://pmc.ncbi.nlm.nih.gov/articles/PMC7233456"
]

## Set up the data fetcher and process the input data

In [5]:
data_gatherer.setup_data_fetcher()

In [6]:
data_resources = data_gatherer.process_urls(input_data)

In [7]:
print(f"Fetched from {len(data_resources)} publications.")
raw_data_files = data_gatherer.classifier.get_raw_data_files(data_resources)
print(f"Fetched {len(raw_data_files)} raw data files:")
raw_data_files

Fetched from 2 publications.
Fetched 3 raw data files:


,publication_url,dataset_id,repository_reference,dataset_webpage
0,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,PHS001049,dbGAP,https://www.ncbi.nlm.nih.gov/projects/gap/cgi-...
1,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,PRJNA306801,SRA,https://www.ncbi.nlm.nih.gov/bioproject/?term=...
2,https://pmc.ncbi.nlm.nih.gov/articles/PMC7233456,phs001287,dbGaP,https://www.ncbi.nlm.nih.gov/projects/gap/cgi-...
